## Run the application on the Production Centre

This Jupyter Notebook queries the catalog for a Sentinel-1 GRD product, creates a Web Processing Service (WPS) request invoking the data transformation application that was deployed in **Step 8**, monitors the WPS request execution and finally retrieves the data transformation execution results

* Set the deployer

In [1]:
deployer = 'https://ec-ecopotential-apps-deployer.terradue.com'

* First do the imports of the Python libraries required

In [3]:
import owslib
from owslib.wps import monitorExecution
import uuid
from owslib.wps import WebProcessingService
import sys
import os
sys.path.append('/opt/anaconda/bin/')
import lxml.etree as etree
os.environ["TMPDIR"] = '1'
os.environ["_CIOP_APPLICATION_PATH"] = '1'
import requests
import cioppy
ciop = cioppy.Cioppy()
import shapely
import dateutil.parser
from shapely.wkt import loads 
import pandas as pd
import geopandas as gp
from datetime import datetime, timedelta
from io import BytesIO
from zipfile import ZipFile
%matplotlib inline
import matplotlib.pyplot as plt
import gdal
import numpy as np

* Define the search parameters: the catalog series OpenSearch endpoint, the time of interest and the area of interest

In [20]:
series = 'https://catalog.terradue.com/sentinel2/search'

start_date = '2017-09-01T00:00:00'
stop_date = '2017-12-10T23:59:59'

geom = 'POLYGON((4.87422567012 43.3610336917, 4.87808277825 43.3593287287, 4.87933078214 43.358984459, 4.8784240356 43.35659545, 4.88064750596 43.35575493, 4.88188374715 43.3560134945, 4.8857850068 43.3561179896, 4.88524908677 43.3587245951, 4.88434641352 43.3638898842, 4.88517662849 43.3633317959, 4.88678190316 43.363372484, 4.89003529789 43.3618231836, 4.89324682366 43.3635473081, 4.89532621202 43.3656197051, 4.89838496263 43.3678747163, 4.90288411884 43.37013327, 4.90969873618 43.3747214536, 4.91295168161 43.3796738878, 4.91249322223 43.3847277718, 4.91598247934 43.3828889721, 4.91586642535 43.376780267, 4.91038249752 43.3719985874, 4.90113645458 43.3655783062, 4.89266341767 43.3595174916, 4.88450091734 43.3534668314, 4.87423348318 43.3466513994, 4.86293715244 43.3386481231, 4.85650120045 43.3284474494, 4.82789349177 43.2850282779, 4.79148463539 43.2895002409, 4.75067696129 43.2995486074, 4.70233955023 43.2982046887, 4.65605192947 43.2966649627, 4.60540936269 43.3047466291, 4.55015978064 43.3177831123, 4.51450560625 43.336326927, 4.49129449343 43.3609129636, 4.48577708902 43.386411243, 4.47320743811 43.405509686, 4.43605279862 43.4001741259, 4.40650799437 43.3991223394, 4.37253225318 43.4020761104, 4.31055379008 43.4065172872, 4.23079067561 43.4538339564, 4.15997534468 43.4788695718, 4.14722434408 43.5122279998, 4.14336233734 43.5413165699, 4.15405931313 43.5514491892, 4.16813429114 43.558194622, 4.16012888991 43.5675783508, 4.18193498647 43.5659587324, 4.19595583757 43.5784875263, 4.16433628654 43.5739022476, 4.16950522142 43.585529224, 4.18290974263 43.5916610879, 4.19254028823 43.6047805737, 4.21104439544 43.6139112385, 4.2128068175 43.637238976, 4.22426918936 43.6564158287, 4.23738826725 43.6682812378, 4.25377641766 43.6684972972, 4.25414511565 43.6556344613, 4.25080770499 43.6340192527, 4.2483664292 43.6162877591, 4.2621644507 43.6245008424, 4.27629350027 43.6366158487, 4.30418771076 43.6401551193, 4.41353669914 43.6512018225, 4.44872042675 43.6679482635, 4.46358721089 43.6689524068, 4.47557585146 43.6716643539, 4.47714447514 43.6800835088, 4.47770851415 43.6890033475, 4.48442982044 43.6973150143, 4.49610667273 43.7011546702, 4.51524258599 43.7023149938, 4.52708921819 43.7032101825, 4.53534600652 43.7070894133, 4.54617398862 43.7048165456, 4.55639846678 43.6975275332, 4.55343711255 43.6913297457, 4.55085871379 43.6855511903, 4.54975226169 43.6800246932, 4.55473185251 43.6729539006, 4.56152241463 43.6693842809, 4.57032551454 43.6665403322, 4.57657981139 43.6658817349, 4.58663406179 43.6673084099, 4.5887051626 43.6641588449, 4.59211101664 43.6593243855, 4.59594170585 43.6515070422, 4.62083042407 43.6426859901, 4.71064179526 43.6358631585, 4.7297426684 43.619072104, 4.68864747538 43.6099462193, 4.67058721563 43.5930562437, 4.70794986295 43.579225543, 4.73382199361 43.569682043, 4.74417615109 43.5599096112, 4.76857192373 43.5570403393, 4.79339441688 43.5431739036, 4.80644730174 43.5240475768, 4.84529401399 43.4879697606, 4.83833237639 43.4690699859, 4.79962079261 43.4379723563, 4.78597575813 43.4192459016, 4.78202323268 43.4008902926, 4.76161806347 43.4053818492, 4.74937677637 43.4147022478, 4.73141261807 43.4159224533, 4.70746843634 43.3959589277, 4.71150410319 43.3715117629, 4.72358629263 43.3576146237, 4.77336583801 43.3797583683, 4.80822380971 43.3813965005, 4.84300981505 43.3783050406, 4.86066177699 43.3715898307, 4.87083498122 43.3651122615, 4.87422567012 43.3610336917))'

* Search for Sentinel-2 products in one of the polygons of the area of interest

In [21]:
wkt = loads(geom).simplify(0.05).wkt

In [22]:
search_params = dict([('geom', wkt),
                     ('start', start_date),
                     ('stop', stop_date),
                      ('pt', 'S2MSI2Ap')])

In [23]:
search = ciop.search(end_point = series,
                     params = search_params,
                     output_fields='self,enclosure,identifier', 
                     model='GeoTime')

* List the Sentinel-2 L2A products found

In [24]:
for index, elem in enumerate(search):
    print(index, elem['identifier'])

(0, 'S2A_MSIL2A_20171209T104431_N0206_R008_T31TEJ_20171209T124737')
(1, 'S2A_MSIL2A_20171206T103421_N0206_R108_T31TEJ_20171206T123751')
(2, 'S2A_MSIL2A_20171206T103421_N0206_R108_T31TFJ_20171206T123751')
(3, 'S2A_MSIL2A_20171206T103421_N0206_R108_T31TFH_20171206T123751')
(4, 'S2A_MSIL2A_20171129T104401_N0206_R008_T31TEJ_20171129T124511')
(5, 'S2A_MSIL2A_20171129T104401_N0206_R008_T31TFH_20171129T124511')
(6, 'S2A_MSIL2A_20171129T104401_N0206_R008_T31TFJ_20171129T124511')
(7, 'S2A_MSIL2A_20171126T103351_N0206_R108_T31TEJ_20171126T122929')
(8, 'S2A_MSIL2A_20171126T103351_N0206_R108_T31TFH_20171126T122929')
(9, 'S2A_MSIL2A_20171126T103351_N0206_R108_T31TFJ_20171126T122929')
(10, 'S2A_MSIL2A_20171119T104331_N0206_R008_T31TFJ_20171120T214946')
(11, 'S2A_MSIL2A_20171119T104331_N0206_R008_T31TEJ_20171120T214946')
(12, 'S2A_MSIL2A_20171119T104331_N0206_R008_T31TFH_20171120T214946')
(13, 'S2A_MSIL2A_20171116T103311_N0206_R108_T31TFJ_20171116T141254')
(14, 'S2A_MSIL2A_20171116T103311_N0206_R108_

* Select the Sentinel-2 L2A product to process

In [25]:
s2_index = 2

* Connect to the WPS server

In [4]:
wps_url = '%s/zoo-bin/zoo_loader.cgi' % deployer

wps = WebProcessingService(wps_url, verbose=False, skip_caps=True)

* Do a GetCapabilities WPS request and list the process:

In [5]:
wps.getcapabilities()

In [6]:
for index, elem in enumerate(wps.processes):
    print(index, elem.identifier)

(0, 'ec_ecopotential_dcs_s1_temporal_series_dcs_s1_temporal_series_0_3')
(1, 'ec_ecopotential_dcs_s2_crop_s2_crop_1_9')
(2, 'coordinator_ec_ecopotential_dcs_s2_crop_s2_crop_1_9')
(3, 'ec_ecopotential_dcs_s2_ndvi_boa_ndvi_boa_1_9')
(4, 'ec_ecopotential_dcs_snap_hydroperiod_preproc_dcs_snap_hydroperiod_preproc_0_6')
(5, 'GetStatus')


* Select the **ec_ecopotential_dcs_s2_ndvi_boa_ndvi_boa_1_9** process and print the title and abstract after having submited a WPS DescribeProcess request

In [7]:
process_id = wps.processes[3].identifier

In [8]:
process = wps.describeprocess(process_id)

In [9]:
process.title

'NDVI BOA - updated 4'

In [10]:
process.abstract

'Sentinel-2 NDVI BOA'

* List the WPS process inputs:

In [11]:
for input in process.dataInputs:
    print(input.identifier)

source
pa_code
pa_name
resolution
plot
flag_expr
_T2Username


* Create a Python dictionary with the inputs:

In [12]:
pa_code = 'CA'
pa_name = 'Camargue'
resolution = '10'
plot = 'False'
flag_expr = 'saturated_l1a_B4'

In [26]:
inputs = [('source', search[s2_index]['self']),
          ('pa_code', pa_code),
          ('pa_name', pa_name),
          ('resolution', resolution),
          ('plot', plot),
          ('flag_expr', flag_expr),
          ('_T2Username', 'fbrito')]

* Submit the Execute WPS request:

In [27]:
execution = owslib.wps.WPSExecution(url=wps.url)

execution_request = execution.buildRequest(process_id, 
                                           inputs, 
                                           output = [('result_osd', False)])

execution_response = execution.submitRequest(etree.tostring(execution_request))

execution.parseResponse(execution_response)

* Monitor the request:

In [28]:
execution.statusLocation

'http://ec-ecopotential-apps-deployer.terradue.com/zoo-bin/zoo_loader.cgi?request=Execute&service=WPS&version=1.0.0&Identifier=GetStatus&DataInputs=sid=71e3afac-7535-11e8-bb2c-0242ac11000e&RawDataOutput=Result'

In [29]:
monitorExecution(execution)

* Check the outcome of the processing request

In [30]:
execution.isSucceded()

True

* Search for the results produced

In [29]:
for output in execution.processOutputs:
    print(output.identifier)

result_osd


In [30]:
results_osd = execution.processOutputs[0].reference

In [33]:
results_osd

In [32]:
search_results = ciop.search(end_point=results_osd,
                         params=[],
                         output_fields='identifier,enclosure',
                         model='GeoTime')

TypeError: execv() arg 2 must contain only strings

In [ ]:
for index, elem in enumerate(search_results):
    print(elem['enclosure'])